In [3]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import KNNWithMeans

# 1. Define the dataset (as provided)
ratings_dict = { 
    "book": ["Book A", "Book B", "Book A", "Book C", "Book B", "Book C", "Book A", "Book B", "Book C"], 
    "user": ['Alice', 'Alice', 'Bob', 'Bob', 'Charlie', 'Charlie', 'David', 'David', 'Eva'], 
    "rating": [4.5, 5, 3.5, 4, 2.5, 3, 4.5, 4, 3.5]
}

df = pd.DataFrame(ratings_dict)

# 2. Define the reader and load the dataset using Surprise
reader = Reader(rating_scale=(1, 5)) 
data = Dataset.load_from_df(df[["user", "book", "rating"]], reader)

# 3. Setup item-based collaborative filtering using KNN
sim_options = {     
    "name": "cosine",         # Cosine similarity
    "user_based": False,      # Item-based filtering
} 
algo = KNNWithMeans(sim_options=sim_options)

# 4. Build the training set and train the model
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)

# 5. Predict ratings for items that the user hasn't rated yet
def recommend_books(user_name, num_recommendations=5):
    # Map user names to internal user IDs
    user_inner_id = trainingSet.to_inner_uid(user_name)
    
    # Get the list of books the user has rated
    rated_books = [item for (item, _) in trainingSet.ur[user_inner_id]]
    
    # Generate predictions for all books the user has not rated
    predictions = []
    for book in trainingSet.all_items():
        if book not in rated_books:
            prediction = algo.predict(user_name, trainingSet.to_raw_iid(book))
            predictions.append((book, prediction.est))
    
    # Sort predictions by estimated rating in descending order
    recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)
    
    # Return the top N recommendations
    return recommendations[:num_recommendations]

# 6. Recommend books for user 'Eva' (or any other user)
user_name = 'Eva'
recommendations = recommend_books(user_name, 5)

# 7. Display the top 5 recommendations
print(f"Top 5 book recommendations for user {user_name}:")
for book, rating in recommendations:
    print(f"Book: {book}, Predicted Rating: {rating:.2f}")


Computing the cosine similarity matrix...
Done computing similarity matrix.
Top 5 book recommendations for user Eva:
Book: 0, Predicted Rating: 4.17
Book: 1, Predicted Rating: 3.83
